# CHAPTER 2 - The Tools of the Trade in Quantum Computing - PennyLane Code

*Note*: You may skip the following three cells if you have alredy installed the right versions of all the libraries mentioned in *Appendix D*. This will likely NOT be the case if you are running this notebook on a cloud service such as Google Colab.



In [ ]:
pip install pennylane==0.26

In [ ]:
pip install qiskit==0.39.2

In [ ]:
pip install pennylane-qiskit==0.24.0

In [1]:
import pennylane as qml

In [5]:
#creo un dispositivo con 2 qubits, è ciò che eseguirò nel mio circuito quantistico
dev = qml.device('default.qubit', wires = 2)

In [6]:
#definisco il mio circuito quantistico,metto un operatore di Pauli X e Hadamard sul primo qubit
def qc():
    qml.PauliX(wires = 0)
    qml.Hadamard(wires = 0)
    return qml.state()

In [7]:
qcirc = qml.QNode(qc, dev) # Assemble the circuit & the device, note the order of arguments->circuit, device
qcirc() # Run it!

array([ 0.70710678+0.j,  0.        +0.j, -0.70710678+0.j,  0.        +0.j])

In [8]:
@qml.qnode(dev) # We add this decorator to use the device dev.
def qcirc():
    qml.PauliX(wires = 0)
    qml.Hadamard(wires = 0)
    return qml.state()
    
# Now qcirc is already a QNode. We can just run it!
qcirc()

array([ 0.70710678+0.j,  0.        +0.j, -0.70710678+0.j,  0.        +0.j])

In [9]:
dev = qml.device('default.qubit', wires = 1)
@qml.qnode(dev)
def qcirc(theta):
    qml.RX(theta, wires = 0)
    return qml.state()

In [10]:
print(qml.draw(qcirc)(theta = 2))

0: ──RX(2.00)─┤  State


In [11]:
dev = qml.device('default.qubit', wires = 2)
@qml.qnode(dev)
def qcirc(theta,phi):
    qml.RX(theta, wires = 0)
    qml.RY(phi, wires = 1)
    return qml.state()

In [12]:
print(qml.draw(qcirc)(theta = 2,phi=1))

0: ──RX(2.00)─┤  State
1: ──RY(1.00)─┤  State


In [15]:
dev = qml.device('default.qubit', wires = 3)

# Get probabilities
@qml.qnode(dev)
def qcirc():
    qml.Hadamard(wires = 1)
    return qml.probs(wires = [1, 2]) # Only the last 2 wires.
prob = qcirc()
print("Probs. wires [1, 2] with H in wire 1:", prob)#Probs. wires [1, 2] with H in wire 1:[0.5 0.  0.5 0. ]->50% 10 e 50% 00

# Get a sample, not having specified shots in the device.
@qml.qnode(dev)
def qcirc():
    qml.Hadamard(wires = 0)
    return qml.sample(wires = 0) # Only the first wire.
s1 = qcirc(shots = 4) # We specify the shots here.
print("Sample 1 after H:", s1)

# Get a sample with shots in the device.
dev = qml.device('default.qubit', wires = 2, shots = 4)
@qml.qnode(dev)
def qcirc():
    qml.Hadamard(wires=0)
    return qml.sample() # Will sample all wires.
s2 = qcirc()
print("Sample 2 after H x I:", s2)

Probs. wires [1, 2] with H in wire 1: [[1 0]
 [1 0]
 [0 0]
 [1 0]]
Sample 1 after H: [[1]
 [1]
 [1]
 [1]]
Sample 2 after H x I: [[1 0]
 [0 0]
 [0 0]
 [0 0]]


c:\Users\vital\source\repos\Vitalessandro\A-Practical-Guide-to-Quantum-Machine-Learning-and-Quantum-Optimization\venv\Lib\site-packages\pennylane\workflow\qnode.py:808: PennyLaneDeprecationWarning: Specifying 'shots' when executing a QNode is deprecated and will be removed in v0.44. Please set shots on QNode initialization, or use qml.set_shots instead.
  shots = self._get_shots(kwargs)
c:\Users\vital\source\repos\Vitalessandro\A-Practical-Guide-to-Quantum-Machine-Learning-and-Quantum-Optimization\venv\Lib\site-packages\pennylane\devices\device_api.py:201: PennyLaneDeprecationWarning: Setting shots on device is deprecated. Please use the `set_shots` transform on the respective QNode instead.
  warnings.warn(


In [13]:
dev = qml.device('qiskit.aer', wires = 2)
@qml.qnode(dev)
def qcirc():
    qml.Hadamard(wires = 0)
    return qml.probs(wires = 0)
s = qcirc()
print("The probabilities are", s)

c:\Users\vital\source\repos\Vitalessandro\A-Practical-Guide-to-Quantum-Machine-Learning-and-Quantum-Optimization\venv\Lib\site-packages\pennylane_qiskit\qiskit_device_legacy.py:144: UserWarning: The analytic calculation of expectations, variances and probabilities is only supported on statevector backends, not on the aer_simulator. Such statistics obtained from this device are estimates based on samples.
  warnings.warn(self.analytic_warning_message.format(self.backend_name), UserWarning)


The probabilities are [0.51953125 0.48046875]


*Note*: In the following cell, you need to replace "1234" with your actual IBM token. Refer to *Appendix D* in the book for instructions on how to create an account and get your token. Be very careful not to disclose your token to anyone!

In [ ]:
import os
ibm_token = os.getenv("IBM_QUANTUM_TOKEN")

Token trovato? lGwOmV6RHZuJ3h_lEUm6OSbcS4X2zqBxTk3dKenIMDxt


In [3]:
from qiskit_ibm_runtime import QiskitRuntimeService

# --- Run ONCE to save credentials locally (then you can remove this line) ---
QiskitRuntimeService.save_account(
    channel="ibm_quantum_platform",
    token=ibm_token,
    overwrite=True,
    set_as_default=True,
)

# Load saved credentials
service = QiskitRuntimeService()

# Pick a real device (least busy non-simulator)
backend = service.least_busy(simulator=False, operational=True)

# PennyLane device uses a Qiskit backend OBJECT (not a backend name string)
dev = qml.device("qiskit.remote", wires=1, backend=backend)

# Send a circuit and get some results!
@qml.qnode(dev)
def qcirc():
    qml.Hadamard(wires = 0)
    return qml.probs(wires = 0)
print(qcirc())

qiskit_runtime_service.__init__:WARNING:2026-01-25 11:54:26,639: Instance was not set at service instantiation. Free and trial plan instances will be prioritized. Based on the following filters: (tags: None, region: us-east, eu-de), and available plans: (open), the available account instances are: open-instance. If you need a specific instance set it explicitly either by using a saved account with a saved default instance or passing it in directly to QiskitRuntimeService().
qiskit_runtime_service.backends:WARNING:2026-01-25 11:54:26,979: Loading instance: open-instance, plan: open
qiskit_runtime_service.backends:WARNING:2026-01-25 11:54:29,650: Using instance: open-instance, plan: open
c:\Users\vital\source\repos\Vitalessandro\A-Practical-Guide-to-Quantum-Machine-Learning-and-Quantum-Optimization\venv\Lib\site-packages\pennylane_qiskit\qiskit_device.py:62: UserWarning: Expected an integer number of shots, but received shots=None. A default number of shots will be selected by the Qiskit

[0.48925781 0.51074219]
